# Network Analysis
This notebook compute cumulative biogas potential for the pipeline segments. 
1. Read in the edge list computed from the pipeline segments. This includes the edge ID, the downstream edge ID, and the amount of biogas potential at the edge source. 
2. Build a graph and then iterate through node on the graph, trace downstream, and tally cumulative downstream biogas potential values.
3. Export as a csv and shapefile

In [1]:
#Import packages
import networkx as nx
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
#Read the edge list in as a dataframe
edgelist_fn = '../data/processed/BaseEdgeList.csv'
df_edgeList = pd.read_csv(edgelist_fn,
                          header=0,
                          names = ['Source_node','To_node','BG_Potential'],
                          dtype={'Source_node':str,'To_node':str,'BG_Potential':np.float64})

In [3]:
#Replace null values in biogas potential with zero
df_edgeList.loc[pd.isna(df_edgeList['BG_Potential']),'BG_Potential'] = 0

#### Create and populate the graph
* Create a multi-directional graph
* Iterate through the edge list and add add edges from the source/downstream nodes & weights

In [4]:
#Create the graph
G = nx.MultiDiGraph()

In [5]:
#Add the edges to the graph
for i,r in df_edgeList.iterrows():
    G.add_edge(r['Source_node'],r['To_node'],cost=r['BG_Potential'])

#### Compute total upstream biogas potential 
Iterate through each "from" node and find all its upstream nodes and sum their collective biogas potential values

In [8]:
#Iterate through each to_node
for source_node in df_edgeList['Source_node'].unique():
    #Reset the cost variable
    total_yield = 0
    #Get the downstream nodes for the current node
    down_nodes = nx.ancestors(G,source_node)
    #Append the node itself
    down_nodes.add(source_node)
    #Iterate through the to_node's ancestors
    for down_node in down_nodes:
        #Accumulate the total yield
        total_yield += df_edgeList.query(F"Source_node == '{down_node}'")['BG_Potential'].sum()
    #Update the output table
    df_edgeList.loc[df_edgeList['Source_node'] == source_node,'SumPotential'] = total_yield

##### Export results

In [22]:
#Save edge list to a filename
edgeCSV_filename =  '../data/processed/Biogas_11May2020.csv'
df_edgeList[['Source_node','To_node', 'SumPotential']].to_csv(edgeCSV_filename,index=False)

In [ ]:
df_edgeList['SumPotential'] = df_edgeList['SumPotential'].round(0)

In [20]:
#Read in segments and join cumulative data
gdf_Segments = gpd.read_file('../data/processed/BasePipeline.shp')
gdf_Segments = gdf_Segments.merge(df_edgeList,left_on='edge_ID',right_on='Source_node',how='left')
gdf_Segments.drop(columns=['downstream','BG_potenti','route_id','edge_ID'],axis=1,inplace=True)
gdf_Segments.to_file('../data/processed/CumulativeBG.shp')